In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Nomenclature
- Complete data collection procedure, from init to writing is referred to as a "run".   
- Each step in a run is an "event".  
- Where a run is in the sequence of events is it's "state"
- To move between states requires "triggers"

## States are
* waiting: in between events; triggered by conclusion of steps.  Options while waiting include
    * set parameters (wavelength/steps, integration time, devices to include in the initialization, compression?, write results?, display in real time?)
    * initialize
    * run all
* initializing (complete and between runs): triggered by initialize command (at any time, but warning if initialize already happened) will initialize all devices with checked boxes in GUI. Returns dictionary of device metadata
* collecting: triggered by collection command (at end sets self.data_in_queue = True)
* compressing: triggered by compression commmand (at end sets self.data_ready_to_store = True) 
* writing: triggered by writing command
* fubar: triggered by an error in any of the steps
* interupt: is this a state or a command?  

In [2]:
import os
import sys
import pdb
import numpy as np

#sys.path.append(r'C:\Users\marco viero\Repositories\SPHEREx-lab-tools\SPHEREx-Calibration-Automation\pylab\pylabcal\pylabcald')
#sys.path.append(r'..\pylab')
sys.path.append(r'..\pylab\pylabcal\pylabcald')

import pylab
from pylabcaldlib.utils import parameters
from pylabcaldlib.instruments.repeatedtimer import RepeatedTimer
from pylabcaldlib.instruments.powermaxusb import PowermaxUSB
from pylabcaldlib.instruments.serial_motor_dpy50601 import DPY50601

In [3]:
class SM:
    
    startState = 'waiting'
    init = False
    in_queue = False
    ready_to_store = False
    
    error_status = False
    
    def start(self):
        self.state = self.startState
        self.init_status = self.init
        self.data_in_queue = self.in_queue
        self.data_ready_to_store = self.ready_to_store
        self.errorStatus = self.error_status
        self.errorDict = {}
        self.metadata = {}
        
    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s

        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

In [4]:
class SpectralCalibrationMachine(SM):
    
    def __init__(self):
        print('Start by Initializing')        
        
        self.path = r"C:\Users\marco viero\Repositories\SPHEREx-lab-tools\SPHEREx-Calibration-Automation\pylab\pylabcal\config"
        self.config_file = 'setup.cfg' #  config_file
        self.message_box = []
        self.params = self.get_parameters(os.path.join(self.path,self.config_file))

        #self.errorStatus = self.initialize()
        self.initialize()
        #pdb.set_trace()

    def initialize(self):
        print('Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict')
        
        self.start()        
        
        # Load Thorlabs Laser
        
        # Load Filter Wheels
        
        # Load Data Collection
        
        # Load Powermax
        self.powermax = PowermaxUSB()
        #self.errorDict['powermax'] = self.powermax.error_record 
        
        # Load First Motor Controller
        self.xstage = DPY50601(0)
        
        #pdb.set_trace()
        if np.sum(sum(self.errorDict.values())):
            #self.error_handler()
            self.errorStatus = True
            self.message_box.append('Initialization unsuccessful')
            #return 
        else:
            self.message_box.append('Initialization successful')
            self.init_status = True
            
        return 'waiting', 'Initialized without errors' #errors 
    
    def get_parameters(self, config_path):

        params = parameters.get_params(config_path)

        return params

    def error_handler(self):
        for key, value in self.errorDict.items():
            #pdb.set_trace()
            print(key,':',value)
            #self.errorStatus = True
        
        self.errorStatus = False
        self.message_box.append('Errors Found. Be more specific!')
        return 'waiting', 'Debug Errors' #errors 
    
    def collect_data(self):
        print('Collecting Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_in_queue = True
            
        self.message_box.append('Collecting Data')
        return 'waiting', 'Data Collected' #errors 
    
    def compress_data(self):
        print('Compressing Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
        
        self.message_box.append('Compressing Data as .??')
        return 'waiting', 'Data Compressed' #errors 
    
    def write_data_to_disk(self):
        print('Writing Data to Disk')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
            
        self.message_box.append('Writing Data to Disk')
        return 'waiting', 'Data Written to Disk' #errors 
    
    def generateOutput(self,state):
        if state == 'initializing':
            return self.initialize()
        elif state == 'collecting':
            return self.collect_data()
        elif state == 'compressing':
            return self.compress_data()
        elif state == 'storing':
            return self.write_data_to_disk()
        elif state == 'error':
            return self.error_handler()        
        else:
            return 'wait'
     
    def getNextValues(self, state, dict_in):
        #pdb.set_trace()
        if self.errorStatus == True:
            nextState = 'error'
        elif state == 'waiting' and 'initialize' in dict_in:
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and 'collect' in dict_in:
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and 'compress' in dict_in:
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and 'write_to_disk' in dict_in:
            nextState = 'storing'
        else:
            nextState = 'error'
        return (self.generateOutput(nextState))

In [5]:
scm2 = SpectralCalibrationMachine()

Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...


In [6]:
scm2.transduce([{'initialize': [0,2,4]}])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...


['Initialized without errors']

In [7]:
scm2.transduce(['initialize','collect','compress','write_to_disk'])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Compressing Data
Writing Data to Disk


['Initialized without errors',
 'Data Collected',
 'Data Compressed',
 'Data Written to Disk']

# GUI Developement

In [8]:
import sys
from math import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

In [9]:
class MainWindow(QMainWindow):

    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)
        
        self.state_machine = SpectralCalibrationMachine()
        
        self.setWindowTitle("Spectral Calibration Interface")
        
        layout = QGridLayout()
        
        # Left Side 
        
        self.initialize_button = QPushButton(self)
        self.initialize_button.setText("Initialize")
        layout.addWidget(self.initialize_button, 0, 0)
        
        self.setup_button = QPushButton(self)
        self.setup_button.setText("Setup")
        layout.addWidget(self.setup_button, 1, 0)

        self.automate_box = QCheckBox(self)
        self.automate_box.setText("Automate")
        layout.addWidget(self.automate_box, 1, 1)

        self.start_run_button = QPushButton(self)
        self.start_run_button.setText("Start Run")
        layout.addWidget(self.start_run_button, 2, 0)
                
        wavelength_start = int(self.state_machine.params['monochrometer']['wavelength_start'])
        wavelength_stop = int(self.state_machine.params['monochrometer']['wavelength_stop'])
        wavelength_step = int(self.state_machine.params['monochrometer']['wavelength_step'])
        label = QLabel("Wavelength: ")
        self.wavelength_select = QSpinBox(self)
        self.wavelength_select.setMinimum(wavelength_start)
        self.wavelength_select.setMaximum(wavelength_stop)
        #self.wavelength_select.singletStep(10)
        self.wavelength_select.setValue(200)    
        self.wavelength_select.valueChanged.connect(self.updateLabel)
        #self.start_lambda = QLabel('0', self)
        layout.addWidget(label, 3, 0)
        layout.addWidget(self.wavelength_select, 3, 1)
        
        # Right Side
        
        #self.msg = QMessageBox()
        self.msg = QTextBrowser()
        #self.msg.setDetailedText("The details are as follows:")
        layout.addWidget(self.msg, 0, 2, 4, 3)
        
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        self.retranslateUi()
        
        
    def updateLabel(self, value):

        self.start_lambda.setText(str(value))
        
    def retranslateUi(self):
        
        self.initialize_button.clicked.connect(self.initialize_button_clicked)
        
        self.setup_button.clicked.connect(self.setup_button_clicked)
        
        self.start_run_button.clicked.connect(self.start_run_button_clicked)
        #self.button2.clicked.connect(self.initialize_button_clicked)
        
        
        self.dialogTextBrowser = setupWindow(self)
        
        #pdb.set_trace()
        if len(self.state_machine.message_box) > 0:
            for i in self.state_machine.message_box:
                last_line = self.state_machine.message_box.pop()
                self.msg.append(last_line)
        
    def initialize_button_clicked(self):
        
        self.state_machine = SpectralCalibrationMachine()

        
    @pyqtSlot()
    def setup_button_clicked(self):
        self.dialogTextBrowser.exec_()

        
    def start_run_button_clicked(self):
        
        state_machine.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

        
class setupWindow(QDialog):
    def __init__(self, obj_in, parent=None):
        super(setupWindow, self).__init__(parent)

        self.setWindowTitle("Settings")

        layout = QGridLayout()
        
        self.metadata_button = QPushButton(self)
        self.metadata_button.setText("Metadata to Include")
        layout.addWidget(self.metadata_button, 0, 0)
        self.metadata_button.clicked.connect(self.metadata_button_clicked)
        
        label = QLabel("Wavelengths: ")
        label_l = QLabel("Start ")
        label_m = QLabel("End ")
        label_r = QLabel("Step ")
        layout.addWidget(label_l, 1, 1)
        layout.addWidget(label_m, 1, 2)
        layout.addWidget(label_r, 1, 3)
        #pdb.set_trace()
        wavelength_start = int(obj_in.state_machine.params['monochrometer']['wavelength_start'])
        wavelength_stop = int(obj_in.state_machine.params['monochrometer']['wavelength_stop'])
        wavelength_step = int(obj_in.state_machine.params['monochrometer']['wavelength_step'])
        self.wavelength0 = QSpinBox(self)
        self.wavelength0.setMinimum(wavelength_start)
        self.wavelength0.setMaximum(wavelength_stop)
        self.wavelength0.setValue(wavelength_start) 
        self.wavelength1 = QSpinBox(self)
        self.wavelength1.setMinimum(wavelength_start)
        self.wavelength1.setMaximum(wavelength_stop)
        self.wavelength1.setValue(stop) 
        self.wavelength2 = QSpinBox(self)
        self.wavelength2.setMinimum(wavelength_start)
        self.wavelength2.setMaximum(wavelength_stop)
        self.wavelength2.setValue(wavelength_step) 
        layout.addWidget(label, 2, 0)
        layout.addWidget(self.wavelength0, 2, 1)
        layout.addWidget(self.wavelength1, 2, 2)
        layout.addWidget(self.wavelength2, 2, 3)
        
        label_write = QLabel("Data Format: ")
        self.csv_box = QCheckBox(self)
        self.csv_box.setText(".csv")
        self.json_box = QCheckBox(self)
        self.json_box.setText(".json")
        self.parquet_box = QCheckBox(self)
        self.parquet_box.setText(".parquet")
        layout.addWidget(label_write, 3, 0)
        layout.addWidget(self.csv_box, 3, 1)
        layout.addWidget(self.json_box, 3, 2)
        layout.addWidget(self.parquet_box, 3, 3)
        
        # calling the uncheck method if any check box state is changed 
        self.csv_box.stateChanged.connect(self.uncheck_write) 
        self.json_box.stateChanged.connect(self.uncheck_write) 
        self.parquet_box.stateChanged.connect(self.uncheck_write) 
        
        label_comp = QLabel("Compression: ")
        self.zip_box = QCheckBox(self)
        self.zip_box.setText(".zip")
        self.tgz_box = QCheckBox(self)
        self.tgz_box.setText(".tgz")
        self.tar_box = QCheckBox(self)
        self.tar_box.setText(".tar")
        layout.addWidget(label_comp, 4, 0)
        layout.addWidget(self.zip_box, 4, 1)
        layout.addWidget(self.tgz_box, 4, 2)
        layout.addWidget(self.tar_box, 4, 3)
        
        # calling the uncheck method if any check box state is changed 
        self.zip_box.stateChanged.connect(self.uncheck_comp) 
        self.tar_box.stateChanged.connect(self.uncheck_comp) 
        self.tgz_box.stateChanged.connect(self.uncheck_comp) 
        
        label_suffix = QLabel("Custom Suffix: ")
        layout.addWidget(label_suffix, 5, 0)
        self.suffix = QLineEdit("_default")
        self.suffix.selectAll()
        layout.addWidget(self.suffix, 5, 1, 1, 3)
        
        self.widget = QWidget()
        self.widget.setLayout(layout)
        #self.setCentralWidget(widget)
        
        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.metadata_button)
        self.verticalLayout.addWidget(self.widget)
        self.verticalLayout.addWidget(self.buttonBox)        
        self.horizontalLayout = QHBoxLayout(self)
        
        self.dialogTextBrowser = metadataWindow(self)

    @pyqtSlot()
    def metadata_button_clicked(self):
        self.dialogTextBrowser.exec_()
        
    # uncheck method 
    def uncheck_comp(self, state): 
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.zip_box: 
  
                # making other check box to uncheck 
                self.tgz_box.setChecked(False) 
                self.tar_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.tar_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tgz_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.tgz_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tar_box.setChecked(False) 
            
    # uncheck method 
    def uncheck_write(self, state): 
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.csv_box: 
  
                # making other check box to uncheck 
                self.json_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.json_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.parquet_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.json_box.setChecked(False) 
            
class metadataWindow(QDialog):
    def __init__(self, parent=None):
        super(metadataWindow, self).__init__(parent)

        self.setWindowTitle("Metadata to Include")
        layout = QGridLayout()

        self.temperature_box = QCheckBox(self)
        self.temperature_box.setText("Cryo_Temperature")
        self.temperature_box.setChecked(True) 
        layout.addWidget(self.temperature_box, 1, 0)

        self.power_meter_box = QCheckBox(self)
        self.power_meter_box.setText("Power_Meter")
        self.power_meter_box.setChecked(True) 
        layout.addWidget(self.power_meter_box, 2, 0)

        self.filter_box = QCheckBox(self)
        self.filter_box.setText("Filters")
        self.filter_box.setChecked(True) 
        layout.addWidget(self.filter_box, 3, 0)
        
        # Keywords
        label_keywords = QLabel("Search Keywords: ")
        layout.addWidget(label_keywords, 0, 0)
        self.suffix = QLineEdit("Test, Development")
        self.suffix.selectAll()
        layout.addWidget(self.suffix, 0, 1, 1, 3)
        
        self.widget = QWidget()
        self.widget.setLayout(layout)
        
        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.widget)
        self.verticalLayout.addWidget(self.buttonBox)   

In [ ]:
app = QApplication(sys.argv)
#form = Form()
form = MainWindow()
form.show()
app.exec_()

Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
